In [1]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob #glob 경로명 지정
import numpy as np

In [2]:
#데이터 불러오기
root_dir="C:/Users/cloud/Anaconda_src/data/kfood/"

#카테고리 지정
categories=["Chicken","Kimchi"]

nb_class=len(categories)

In [3]:
#이미지 크기 지정

img_w=64; img_h=64;

In [4]:
## 데이터 변수

X=[] #이미지 데이터
Y=[] #레이블 데이터

for idx, cat in enumerate(categories):
  img_dir=root_dir +cat
  files=glob.glob(img_dir+"/"+"*.jpg")
  print(img_dir+"/"+"*.jpg")
  
  #이미지 로딩
  for i, f in enumerate(files):
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((img_w,img_h))
    data=np.asarray(img)
    X.append(data)
    Y.append(idx)
  
X=np.array(X)
Y=np.array(Y)
print(X.shape,Y.shape)

C:/Users/cloud/Anaconda_src/data/kfood/Chicken/*.jpg
C:/Users/cloud/Anaconda_src/data/kfood/Kimchi/*.jpg
(60, 64, 64, 3) (60,)


In [5]:
#데이터 셋 나누기

X_train,X_test, y_train, y_test= train_test_split(X,Y)
xy=(X_train, X_test, y_train, y_test)

In [6]:
#파일 저장
np.save(root_dir+"kfood02.npy",xy)

### keras를 이용하여 분류모델 만들기

In [7]:
import sys,os
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [8]:
#데이터 불러오기
root_dir="C:/Users/cloud/Anaconda_src/data/kfood/"

#카테고리 지정
categories=["Chicken","Kimchi"]
nb_class=len(categories)

#이미지 크기 저장
img_size=64

In [9]:
#데이터 불러오기
def load_dataset():
    x_train,x_test,y_train,y_test=np.load(root_dir+"kfood02.npy", allow_pickle=True)
    x_train=x_train.astype("float")/256
    x_test=x_test.astype("float")/256
    y_train=np_utils.to_categorical(y_train, nb_class)
    y_test=np_utils.to_categorical(y_test,nb_class)
    
    return x_train, x_test, y_train, y_test

In [10]:
#모델구성 

### 3 layers
## filter 3 by 3
## Activation function is relu
## Maxpool 2 by 2
## dropout 0.25


def build_model(in_shape):
    model=Sequential()
    model.add(Convolution2D(32,3,3,border_mode="Same",
                           input_shape=in_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64,3,3,border_mode="Same",
                           input_shape=in_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,3,3,border_mode="Same",
                           input_shape=in_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())  #한층으로 쌓기
    model.add(Dense(512))  #512층으로 만들기
    model.add(Activation("relu"))
    
    model.add(Dense(nb_class))
    model.add(Activation("softmax"))
    model.compile(loss="binary_crossentropy",
                 optimizer="rmsprop",
                 metrics=["accuracy"])
    
    return model

In [28]:
#모델 학습하기
def model_train(x,y):
    
    model=build_model(x.shape[1:]) #모델만들기
    model.fit(x, y, batch_size=32, epochs=40)
    
    return model

In [12]:
#모델 평가하기
def model_evaluate(model,x,y):
    score=model.evaluate(x,y)
    print("loss = ",score[0])
    print("\n")
    print("accuracy = ",score[1])

In [29]:
X_train, X_test, y_train, y_test=load_dataset() #데이터 불러오기
model=model_train(X_train,y_train) #모델 학습
model_evaluate(model, X_test, y_test) #모델 평가

C:\Python\py\Anaconda\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="Same")`
  del sys.path[0]
C:\Python\py\Anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(64, 64, 3..., padding="Same")`
C:\Python\py\Anaconda\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), input_shape=(64, 64, 3..., padding="Same")`


Epoch 1/40
45/45 [==============================] - 1s 12ms/step - loss: 3.2056 - accuracy: 0.5111
Epoch 2/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 3/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 4/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 5/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 6/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 7/40
45/45 [==============================] - 0s 9ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 8/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 9/40
45/45 [==============================] - 0s 8ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 10/40
45/45 [==============================] - 0s 9ms/step - loss: 7.1769 - accuracy: 0.5333
Epoch 11/40
45/45 

In [30]:
# 모델 저장
model.save(root_dir+"kfood02_model.h5")

In [31]:
## 모델 테스트 하기

import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from PIL import Image
import numpy as np

In [32]:
#데이터 불러오기
root_dir="C:/Users/cloud/Anaconda_src/data/kfood/"

img_files=[root_dir+"test_img/Dolsotbab_16.jpg",
           root_dir+"test_img/kimchi05.jpg",
           root_dir+"test_img/test_chicken02.png"]

#카테고리 지정
categories=["Chicken","Kimchi"]

nb_class=len(categories)

In [33]:
## 이미지 불러오기

X=[]; files=[];

for fname in img_files:
    print(fname)
    img=Image.open(fname)
    img=img.convert("RGB") #색깔
    img= img.resize((img_size, img_size))
    in_data=np.asarray(img)
    in_data= in_data.astype("float") /256
    X.append(in_data)
    files.append(fname)
    
X=np.array(X)    
print(X)
print(files)

C:/Users/cloud/Anaconda_src/data/kfood/test_img/Dolsotbab_16.jpg
C:/Users/cloud/Anaconda_src/data/kfood/test_img/kimchi05.jpg
C:/Users/cloud/Anaconda_src/data/kfood/test_img/test_chicken02.png
[[[[0.16796875 0.08984375 0.125     ]
   [0.1640625  0.10546875 0.1171875 ]
   [0.19140625 0.1015625  0.125     ]
   ...
   [0.265625   0.09375    0.08203125]
   [0.22265625 0.06640625 0.05859375]
   [0.234375   0.07421875 0.05859375]]

  [[0.22265625 0.1015625  0.09375   ]
   [0.2578125  0.1015625  0.09375   ]
   [0.203125   0.0859375  0.078125  ]
   ...
   [0.20703125 0.0703125  0.0625    ]
   [0.25       0.078125   0.06640625]
   [0.2578125  0.08984375 0.06640625]]

  [[0.32421875 0.11328125 0.07421875]
   [0.34765625 0.125      0.09765625]
   [0.2890625  0.08984375 0.07421875]
   ...
   [0.3203125  0.125      0.08984375]
   [0.25       0.0859375  0.08984375]
   [0.234375   0.078125   0.08203125]]

  ...

  [[0.21875    0.0625     0.0234375 ]
   [0.2421875  0.0703125  0.02734375]
   [0.2578125

In [34]:
#파일 읽어오기
model=load_model(root_dir+"kfood02_model.h5")

#모델 예측
pre=model.predict(X)

#예측 출력
for idx, p in enumerate(pre):
    y=p.argmax()
    print("입력 : ",files[idx])
    print("예측 : ", " [", y, "]",
          categories[y], " / score ",p[y])

입력 :  C:/Users/cloud/Anaconda_src/data/kfood/test_img/Dolsotbab_16.jpg
예측 :   [ 1 ] Kimchi  / score  0.9999987
입력 :  C:/Users/cloud/Anaconda_src/data/kfood/test_img/kimchi05.jpg
예측 :   [ 1 ] Kimchi  / score  0.99999976
입력 :  C:/Users/cloud/Anaconda_src/data/kfood/test_img/test_chicken02.png
예측 :   [ 1 ] Kimchi  / score  0.9999999


##### 이미지를 좀 더 많이 학습 시켜서 해봐야 그럴 듯한 모델 완성